In [40]:
import pm4py, os

#import event logs
if __name__ == "__main__":
    log_ec_purchase_order = pm4py.read_xes(os.path.join(preprocessed_dir, 'EC Purchase order.xes'))
    log_framework_purchase_order = pm4py.read_xes(os.path.join(preprocessed_dir, 'Framework order.xes'))
    log_standard_po = pm4py.read_xes(os.path.join(preprocessed_dir, 'Standard PO.xes'))
    
    #infrequent variants filtered out
    log_ec_purchase_order_infVarOut = pm4py.read_xes(os.path.join(preprocessed_dir, 'with filtering out infrequent Variants\\EC Purchase order.xes'))
    log_framework_purchase_order_infVarOut = pm4py.read_xes(os.path.join(preprocessed_dir, 'with filtering out infrequent Variants\\Framework order.xes'))
    log_standard_po_infVarOut = pm4py.read_xes(os.path.join(preprocessed_dir, 'with filtering out infrequent Variants\\Standard PO.xes'))

parsing log, completed traces ::   0%|          | 0/346 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/264 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/8918 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/235 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/116 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/7687 [00:00<?, ?it/s]

In [41]:
#filter out variants, that only consist of events:
#     "Create Purchase Order Item", "Delete Purchase Order Item"
def filter_out_cpoi_dpoi(log):
    return pm4py.filter_variants(log, [["Create Purchase Order Item", "Delete Purchase Order Item"]], retain=False)

#write event log to a xes file in the preprocessed directory
def write_log_to_file(log, filename):
    path = os.path.join(os.path.dirname(os.getcwd()), 'Preprocessed', filename)
    pm4py.write_xes(log, path)

In [42]:
#without infrequent Variants filtered out

# filter out variants with only ("Create Purchase Order Item", "Delete Purchase Order Item")
log_ec_purchase_order = filter_out_cpoi_dpoi(log_ec_purchase_order)
log_framework_purchase_order = filter_out_cpoi_dpoi(log_framework_purchase_order)
log_standard_po = filter_out_cpoi_dpoi(log_standard_po)

# write filtered logs to files
write_log_to_file(log_ec_purchase_order, 'EC Purchase order_preprocessed.xes')
write_log_to_file(log_framework_purchase_order, 'Framework order_preprocessed.xes')
write_log_to_file(log_standard_po, 'Standard PO_preprocessed.xes')

exporting log, completed traces ::   0%|          | 0/346 [00:00<?, ?it/s]

exporting log, completed traces ::   0%|          | 0/232 [00:00<?, ?it/s]

exporting log, completed traces ::   0%|          | 0/8740 [00:00<?, ?it/s]

In [43]:
#with infrequent Variants filtered out

# filter out variants with only ("Create Purchase Order Item", "Delete Purchase Order Item")
log_ec_purchase_order_infVarOut = filter_out_cpoi_dpoi(log_ec_purchase_order_infVarOut)
log_framework_purchase_order_infVarOut = filter_out_cpoi_dpoi(log_framework_purchase_order_infVarOut)
log_standard_po_infVarOut = filter_out_cpoi_dpoi(log_standard_po_infVarOut)

# write filtered logs to files
write_log_to_file(log_ec_purchase_order_infVarOut, 'with filtering out infrequent Variants\\EC Purchase order_preprocessed.xes')
write_log_to_file(log_framework_purchase_order_infVarOut, 'with filtering out infrequent Variants\\Framework order_preprocessed.xes')
write_log_to_file(log_standard_po_infVarOut, 'with filtering out infrequent Variants\\Standard PO_preprocessed.xes')

exporting log, completed traces ::   0%|          | 0/235 [00:00<?, ?it/s]

exporting log, completed traces ::   0%|          | 0/84 [00:00<?, ?it/s]

exporting log, completed traces ::   0%|          | 0/7509 [00:00<?, ?it/s]